In [13]:
import pandas as pd
import twint
import numpy as np
import nest_asyncio
nest_asyncio.apply()
import datetime
import ast
from newspaper import Article
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words('english'))
import re
import swifter
from urllib.parse import urlparse

from tqdm.auto import tqdm
tqdm.pandas()
import requests

In [2]:
df = pd.read_csv("OnlineKhabar_En.csv", parse_dates=['created_at'])

In [3]:
df

,id,created_at,urls,article
0,1377022218178154498,2021-03-31 04:02:35+05:45,https://english.onlinekhabar.com/mithila-house...,"When people talk about Ramayana, one of the gr..."
1,1376853071083413505,2021-03-30 16:50:27+05:45,https://english.onlinekhabar.com/uml-feud-nepa...,"Kathmandu, March 30\n\nThe faction of the ruli..."
2,1376844319739371521,2021-03-30 16:15:41+05:45,https://english.onlinekhabar.com/bara-sexagena...,"Birgunj, March 30\n\nA 65-year-old woman died ..."
3,1376839101039607808,2021-03-30 15:54:56+05:45,https://english.onlinekhabar.com/nepal-covid-1...,"Kathmandu, March 29\n\nThe Ministry of Health ..."
4,1376835218607198211,2021-03-30 15:39:31+05:45,https://english.onlinekhabar.com/78-businesses...,"Kathmandu, March 30\n\nThe Department of Food ..."
...,...,...,...,...
9075,972791011457032193,2018-03-11 16:52:05+05:45,http://english.onlinekhabar.com/kailali-withcr...,"Kathmandu, March 11\n\nThe Mayor of a town in ..."
9076,972785320977752065,2018-03-11 16:29:28+05:45,http://english.onlinekhabar.com/province-5-chi...,"Kathmandu, March 11\n\nProvince 5 Chief Minist..."
9077,972770673549627392,2018-03-11 15:31:16+05:45,http://english.onlinekhabar.com/kailali-withcr...,"Kathmandu, March 11\n\nThe policemen who relea..."
9078,972760525049335808,2018-03-11 14:50:56+05:45,http://english.onlinekhabar.com/icc-world-cup-...,Watching Nepal play in the World Cup is every ...


In [17]:
# # todo change this to func
# def filter_url(url):
#     return ((urlparse(url).netloc)=="www.nepalitimes.com")

In [18]:
# df=df[df.urls.apply(filter_url)]

In [19]:
# def clean_text(raw):
#     text = BeautifulSoup(raw, 'lxml').text
#     word_tokens = word_tokenize(text.lower().rstrip())
#     filtered_sentence = [w for w in word_tokens if not w in stop_words]
#     return ' '.join(filtered_sentence)

In [15]:
def article(url):
    try:
        article = Article(url)
        article.download()
        article.parse()
        return (article.text)
    except Exception as e:
        print (str(e))
        return 404

In [5]:
def primary_url(urls):
    if len(urls)==0:
        return 404
    return urls[0]

In [6]:
def get_new_articles(newspaper_username, since, until):
    c = twint.Config()
    c.Username = newspaper_username
    c.Pandas = True
    c.Hide_output= True
    c.Since = since
    c.Until = until
    twint.run.Search(c)
    df = twint.storage.panda.Tweets_df
    try: 
        df = df[['id', 'created_at', "urls"]]
        df['urls'] = df.urls.apply(primary_url)
        df = df[df.urls!=404]
#         df = df[df.urls.apply(filter_url)]
        df['created_at'] = (pd.to_datetime(df['created_at'], unit='ms')
                         .dt.tz_localize('utc')
                         .dt.tz_convert('Asia/Kathmandu'))   
        return df
    except:
        return pd.DataFrame(columns=['id', 'created_at', "urls"])

In [7]:
data_level1 = [
    'arabl land',
    'avail data',
    'bureau statist',
    'busi survey',
    'cens publi',
    'cens pop',
    'children employ',
    'civil registr',
    'collect method',
    'commerci export',
    'complet rate',
    'consum electr',
    'consum energ',
    'data access',
    'data collect',
    'data compil',
    'data entri',
    'data manag',
    'data releas',
    'data standard',
    'data user',
    'demograph data',
    'densit popul',
    'develop data',
    'difusion dat',
    'direct statist',
    'disaggreg data',
    'electr access',
    'electr consumpt',
    'energi consumpt',
    'establish survey',
    'exchang rate',
    'extern debt',
    'fertil rate',
    'food import',
    'food product',
    'gender gap',
    'govern debt',
    'govern statist',
    'gross domest',
    'gross nation',
    'health expenditur',
    'health survey',
    'import marchandis',
    'improv data',
    'improv statist',
    'indic measur',
    'indic preci',
    'inflat rate',
    'institut statist',
    'interest payment',
    'intern tourism',
    'irrig land',
    'land use',
    'life expect',
    'livestock product',
    'merchandis export',
    'merchandis trade',
    'model statist',
    'mortal rate',
    'multilater debt',
    'nation account',
    'nation statist',
    'nation survey',
    'national brut',
    'national statist',
    'open data',
    'part revenus',
    'pay gap',
    'popul census',
    'popul growth',
    'popul rate',
    'price index',
    'produccion aliment',
    'purchas power',
    'qualiti data',
    'receit fiscal',
    'releas data',
    'revenu fiscal',
    'rural popul',
    'servic export',
    'statist agenc',
    'statist author',
    'statist avail',
    'statist committe',
    'statist data',
    'statist depart',
    'statist national',
    'statist offic',
    'statist servic',
    'statist studi',
    'survey catalogu',
    'tax payment',
    'tax revenu',
    'trade balanc',
    'unemploy rate',
    'use data',
    'water suppli',
    'youth unemploy',
    ]

data_level2 = [
    'accur',
    'adequ',
    'ambigu',
    'apropi',
    'bancal',
    'bias',
    'confiabl',
    'correct',
    'deceit',
    'deceiv',
    'decept',
    'defectu',
    'delud',
    'engan',
    'equivoc',
    'erreur',
    'erro',
    'erron',
    'errone',
    'error',
    'exact',
    'exat',
    'fake',
    'fallaci',
    'faux',
    'fiabl',
    'generaliz',
    'illus',
    'imparcial',
    'impartial',
    'imprecis',
    'improp',
    'inaccur',
    'incorrect',
    'inexact',
    'invalid',
    'limit',
    'manipul',
    'mislead',
    'mistaken',
    'parcial',
    'prec',
    'precis',
    'proper',
    'reliabl',
    'rigor',
    'rigour',
    'scientif',
    'spurious',
    'tromp',
    'trompeur',
    'unbias',
    'unreli',
    'unscientif',
    'unsound',
    'vag',
    'vagu',
    'val',
    'valid',
    ]

data_level3 = [
    'data manipul',
    'lead question',
    'manipul dat',
    'report bias',
    'sampl select',
    'sampl size',
    ]

data_level_indicator = [
    ' cpi ',
    ' fdi ',
    ' gdp ',
    ' gnp ',
    ' hdi ',
    ' wdi ',
    ]

filter_list = [' data ', ' record ', ' research ', ' statistics ',
               ' study ']

In [8]:
def level1_count(article):
    if article:
        article = str(article)
        keyword_list = []
        for word in data_level1:
            search_ = r"\b" + word.split()[0] + r"[a-zA-Z]*\s\b" \
                + word.split()[1] + r"[a-zA-Z]*"
            if re.search(search_, article):
                keyword_list.append(word)
        for word in data_level_indicator:
            if word in article:
                keyword_list.append(word)
        return keyword_list
    return []


def level2_count(article, valid):
    if valid == 1:
        article = str(article)
        keyword_list = []
        for word in data_level2:
            search_ = r"\b" + word + r"[a-zA-Z]*"
            if re.search(search_,  article):
                keyword_list.append(word)
        return keyword_list
    return []


def level3_count(article, valid):
    if valid == 1:
        article = str(article)
        keyword_list = []
        for word in data_level3:
            search_ = r"\b" + word + r"[a-zA-Z]*"
            if re.search(search_,   article):
                keyword_list.append(word)
        return keyword_list
    return []


def level_2_3_filter(article):
    article = str(article)
    for word in filter_list:
        if word in article:
            return 1
    return 0


def level_len(count_list):
    if type(count_list) == str:
        return (1 if len(count_list) > 2 else 0)
    return (1 if len(count_list) > 0 else 0)

In [9]:
df_ = get_new_articles(newspaper_username='OnlineKhabar_En', 
                       since=df.created_at.max().strftime("%Y-%m-%d %H:%M:%S"),
                       until=datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.


<ipython-input-6-bd3985bb40f2>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['urls'] = df.urls.apply(primary_url)


In [10]:
df_

,id,created_at,urls
0,1394067880849805316,2021-05-17 04:55:57+05:45,https://english.onlinekhabar.com/nepal-pm-oli-...
1,1394067475059286017,2021-05-17 04:54:21+05:45,https://english.onlinekhabar.com/covid-19-nepa...
2,1394066855464079360,2021-05-17 04:51:53+05:45,https://english.onlinekhabar.com/nepal-covid-1...
3,1394066499879325699,2021-05-17 04:50:28+05:45,https://english.onlinekhabar.com/tips-maintain...
4,1393889804920639492,2021-05-16 17:08:21+05:45,https://english.onlinekhabar.com/nepal-covid-1...
...,...,...,...
595,1377194837972164608,2021-03-31 15:28:31+05:45,https://english.onlinekhabar.com/chinese-man-h...
596,1377189300643667978,2021-03-31 15:06:30+05:45,https://english.onlinekhabar.com/world-bank-wi...
597,1377178938250653704,2021-03-31 14:25:20+05:45,https://english.onlinekhabar.com/nepal-prepare...
598,1377171651486310404,2021-03-31 13:56:22+05:45,https://english.onlinekhabar.com/iron-mine-ext...


In [16]:
df_['article'] = df_.urls.progress_apply(article)

In [17]:
df_ = df_[df_['article']!=404]

In [18]:
df = df.append(df_)

In [19]:
df

,id,created_at,urls,article
0,1377022218178154498,2021-03-31 04:02:35+05:45,https://english.onlinekhabar.com/mithila-house...,"When people talk about Ramayana, one of the gr..."
1,1376853071083413505,2021-03-30 16:50:27+05:45,https://english.onlinekhabar.com/uml-feud-nepa...,"Kathmandu, March 30\n\nThe faction of the ruli..."
2,1376844319739371521,2021-03-30 16:15:41+05:45,https://english.onlinekhabar.com/bara-sexagena...,"Birgunj, March 30\n\nA 65-year-old woman died ..."
3,1376839101039607808,2021-03-30 15:54:56+05:45,https://english.onlinekhabar.com/nepal-covid-1...,"Kathmandu, March 29\n\nThe Ministry of Health ..."
4,1376835218607198211,2021-03-30 15:39:31+05:45,https://english.onlinekhabar.com/78-businesses...,"Kathmandu, March 30\n\nThe Department of Food ..."
...,...,...,...,...
595,1377194837972164608,2021-03-31 15:28:31+05:45,https://english.onlinekhabar.com/chinese-man-h...,"Kathmandu, March 31\n\nPolice in Kathmandu hav..."
596,1377189300643667978,2021-03-31 15:06:30+05:45,https://english.onlinekhabar.com/world-bank-wi...,"Kathmandu, March 31\n\nThe World Bank has proj..."
597,1377178938250653704,2021-03-31 14:25:20+05:45,https://english.onlinekhabar.com/nepal-prepare...,"Kathmandu, March 31\n\nThe government of Nepal..."
598,1377171651486310404,2021-03-31 13:56:22+05:45,https://english.onlinekhabar.com/iron-mine-ext...,"Baitadi, March 31\n\nThe extraction of an iron..."


In [20]:
# df['level1'] = np.nan
# df['level2'] = np.nan
# df['level3'] = np.nan

In [21]:
df

,id,created_at,urls,article,level1,level2,level3
0,1377022218178154498,2021-03-31 04:02:35+05:45,https://english.onlinekhabar.com/mithila-house...,"When people talk about Ramayana, one of the gr...",NaN,NaN,NaN
1,1376853071083413505,2021-03-30 16:50:27+05:45,https://english.onlinekhabar.com/uml-feud-nepa...,"Kathmandu, March 30\n\nThe faction of the ruli...",NaN,NaN,NaN
2,1376844319739371521,2021-03-30 16:15:41+05:45,https://english.onlinekhabar.com/bara-sexagena...,"Birgunj, March 30\n\nA 65-year-old woman died ...",NaN,NaN,NaN
3,1376839101039607808,2021-03-30 15:54:56+05:45,https://english.onlinekhabar.com/nepal-covid-1...,"Kathmandu, March 29\n\nThe Ministry of Health ...",NaN,NaN,NaN
4,1376835218607198211,2021-03-30 15:39:31+05:45,https://english.onlinekhabar.com/78-businesses...,"Kathmandu, March 30\n\nThe Department of Food ...",NaN,NaN,NaN
...,...,...,...,...,...,...,...
595,1377194837972164608,2021-03-31 15:28:31+05:45,https://english.onlinekhabar.com/chinese-man-h...,"Kathmandu, March 31\n\nPolice in Kathmandu hav...",NaN,NaN,NaN
596,1377189300643667978,2021-03-31 15:06:30+05:45,https://english.onlinekhabar.com/world-bank-wi...,"Kathmandu, March 31\n\nThe World Bank has proj...",NaN,NaN,NaN
597,1377178938250653704,2021-03-31 14:25:20+05:45,https://english.onlinekhabar.com/nepal-prepare...,"Kathmandu, March 31\n\nThe government of Nepal...",NaN,NaN,NaN
598,1377171651486310404,2021-03-31 13:56:22+05:45,https://english.onlinekhabar.com/iron-mine-ext...,"Baitadi, March 31\n\nThe extraction of an iron...",NaN,NaN,NaN


In [22]:
df['level1'] = df.apply(lambda x: (level1_count(x['article']) if pd.isnull(x.level1) else x.level1), axis=1)

In [23]:
df['level1_count'] = df.level1.apply(level_len)

In [24]:
df['level_2_3_valid'] = df['article'].apply(level_2_3_filter)

In [25]:
df['level2'] = df.apply(lambda x: (level2_count(x['article'
                                    ], x['level_2_3_valid'
                                    ]) if pd.isnull(x.level2) else x.level2),
                                    axis=1)


In [26]:
df['level2_count'] = df.level2.apply(level_len)

In [27]:
df['level3'] = df.apply(lambda x: (level3_count(x['article'
                                    ], x['level_2_3_valid'
                                    ]) if pd.isnull(x.level3) else x.level3),
                                    axis=1)

In [28]:
df['level3_count'] = df.level3.apply(level_len)

In [29]:
df.level1_count.sum()/df.shape[0]

0.019632155404009092

In [30]:
df.level2_count.sum()/df.level_2_3_valid.sum()

0.5597609561752988

In [31]:
df.level3_count.sum()/df.level_2_3_valid.sum()

0.0

In [32]:
df.to_csv('OnlineKhabar_En_analysis.csv', index=False)

In [33]:
df.created_at.max()

Timestamp('2021-05-17 04:55:57+0545', tz='Asia/Kathmandu')

In [34]:
df

,id,created_at,urls,article,level1,level2,level3,level1_count,level_2_3_valid,level2_count,level3_count
0,1377022218178154498,2021-03-31 04:02:35+05:45,https://english.onlinekhabar.com/mithila-house...,"When people talk about Ramayana, one of the gr...",[],[],[],0,0,0,0
1,1376853071083413505,2021-03-30 16:50:27+05:45,https://english.onlinekhabar.com/uml-feud-nepa...,"Kathmandu, March 30\n\nThe faction of the ruli...",[],[],[],0,0,0,0
2,1376844319739371521,2021-03-30 16:15:41+05:45,https://english.onlinekhabar.com/bara-sexagena...,"Birgunj, March 30\n\nA 65-year-old woman died ...",[],[],[],0,0,0,0
3,1376839101039607808,2021-03-30 15:54:56+05:45,https://english.onlinekhabar.com/nepal-covid-1...,"Kathmandu, March 29\n\nThe Ministry of Health ...",[],[],[],0,0,0,0
4,1376835218607198211,2021-03-30 15:39:31+05:45,https://english.onlinekhabar.com/78-businesses...,"Kathmandu, March 30\n\nThe Department of Food ...",[],[],[],0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
595,1377194837972164608,2021-03-31 15:28:31+05:45,https://english.onlinekhabar.com/chinese-man-h...,"Kathmandu, March 31\n\nPolice in Kathmandu hav...",[],[],[],0,0,0,0
596,1377189300643667978,2021-03-31 15:06:30+05:45,https://english.onlinekhabar.com/world-bank-wi...,"Kathmandu, March 31\n\nThe World Bank has proj...",[],[],[],0,0,0,0
597,1377178938250653704,2021-03-31 14:25:20+05:45,https://english.onlinekhabar.com/nepal-prepare...,"Kathmandu, March 31\n\nThe government of Nepal...",[],[],[],0,0,0,0
598,1377171651486310404,2021-03-31 13:56:22+05:45,https://english.onlinekhabar.com/iron-mine-ext...,"Baitadi, March 31\n\nThe extraction of an iron...",[],[],[],0,0,0,0


In [37]:
df[['urls',
 'created_at',
 'level1_count',
 'level_2_3_valid',
 'level2_count',
 'level3_count']].to_csv("OnlineKhabar_En_flask.csv", index=False)

In [38]:
df

,id,created_at,urls,article,level1,level2,level3,level1_count,level_2_3_valid,level2_count,level3_count
0,1377022218178154498,2021-03-31 04:02:35+05:45,https://english.onlinekhabar.com/mithila-house...,"When people talk about Ramayana, one of the gr...",[],[],[],0,0,0,0
1,1376853071083413505,2021-03-30 16:50:27+05:45,https://english.onlinekhabar.com/uml-feud-nepa...,"Kathmandu, March 30\n\nThe faction of the ruli...",[],[],[],0,0,0,0
2,1376844319739371521,2021-03-30 16:15:41+05:45,https://english.onlinekhabar.com/bara-sexagena...,"Birgunj, March 30\n\nA 65-year-old woman died ...",[],[],[],0,0,0,0
3,1376839101039607808,2021-03-30 15:54:56+05:45,https://english.onlinekhabar.com/nepal-covid-1...,"Kathmandu, March 29\n\nThe Ministry of Health ...",[],[],[],0,0,0,0
4,1376835218607198211,2021-03-30 15:39:31+05:45,https://english.onlinekhabar.com/78-businesses...,"Kathmandu, March 30\n\nThe Department of Food ...",[],[],[],0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
595,1377194837972164608,2021-03-31 15:28:31+05:45,https://english.onlinekhabar.com/chinese-man-h...,"Kathmandu, March 31\n\nPolice in Kathmandu hav...",[],[],[],0,0,0,0
596,1377189300643667978,2021-03-31 15:06:30+05:45,https://english.onlinekhabar.com/world-bank-wi...,"Kathmandu, March 31\n\nThe World Bank has proj...",[],[],[],0,0,0,0
597,1377178938250653704,2021-03-31 14:25:20+05:45,https://english.onlinekhabar.com/nepal-prepare...,"Kathmandu, March 31\n\nThe government of Nepal...",[],[],[],0,0,0,0
598,1377171651486310404,2021-03-31 13:56:22+05:45,https://english.onlinekhabar.com/iron-mine-ext...,"Baitadi, March 31\n\nThe extraction of an iron...",[],[],[],0,0,0,0
